In [11]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: [Mar 24, 11:59 pm](https://github.com/UBC-CS/cpsc330-2024W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [12]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/miniconda3/envs/cpsc330/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/miniconda3/envs/cpsc330/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/miniconda3/envs/cpsc330/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
 

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W2/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
# Word 1: technology
similar_to_technology = glove_wiki_vectors.most_similar("technology", topn=10)
print("Words similar to 'technology':")
for word, score in similar_to_technology:
    print(f"{word}: {score:.4f}")

Words similar to 'technology':
technologies: 0.8506
computer: 0.7642
tech: 0.7489
software: 0.7359
systems: 0.7293
innovation: 0.7234
technological: 0.7218
engineering: 0.7113
research: 0.7066
computing: 0.7048


In [7]:
# Word 2: university
similar_to_university = glove_wiki_vectors.most_similar("university", topn=10)
print("Words similar to 'university':")
for word, score in similar_to_university:
    print(f"{word}: {score:.4f}")

Words similar to 'university':
college: 0.8294
harvard: 0.8156
yale: 0.8114
professor: 0.8104
graduate: 0.7993
faculty: 0.7793
princeton: 0.7627
institute: 0.7604
school: 0.7548
stanford: 0.7425


In [8]:
# Word 3: happy
similar_to_happy = glove_wiki_vectors.most_similar("happy", topn=10)
print("Words similar to 'happy':")
for word, score in similar_to_happy:
    print(f"{word}: {score:.4f}")

Words similar to 'happy':
'm: 0.8413
feel: 0.8133
're: 0.8048
i: 0.7938
'll: 0.7916
really: 0.7904
glad: 0.7833
good: 0.7822
we: 0.7809
sure: 0.7788


In [9]:
# Word 4: programming
similar_to_programming = glove_wiki_vectors.most_similar("programming", topn=10)
print("Words similar to 'programming':")
for word, score in similar_to_programming:
    print(f"{word}: {score:.4f}")

Words similar to 'programming':
network: 0.7121
format: 0.7066
broadcast: 0.7060
channels: 0.6912
programs: 0.6877
channel: 0.6870
broadcasts: 0.6868
television: 0.6730
content: 0.6700
tv: 0.6638


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [10]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [12]:
results = []

for word1, word2 in word_pairs:
    try:
        similarity = glove_wiki_vectors.similarity(word1, word2)
        results.append((word1, word2, similarity))
    except KeyError as e:
        results.append((word1, word2, f"Error: {e}"))

similarity_df = pd.DataFrame(results, columns=["Word 1", "Word 2", "Cosine Similarity"])
similarity_df

,Word 1,Word 2,Cosine Similarity
0,coast,shore,0.700027
1,clothes,closet,0.546276
2,old,new,0.643249
3,smart,intelligent,0.755273
4,dog,cat,0.879807
5,tree,lawyer,0.076719


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [13]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [14]:
results = []

for word in test_words:
    exists = word in glove_wiki_vectors
    results.append((word, exists))

word_exists_df = pd.DataFrame(results, columns=["Word", "Exists in Vocabulary"])
word_exists_df

,Word,Exists in Vocabulary
0,covididiot,False
1,fomo,False
2,frenemies,True
3,anthropause,False
4,photobomb,False
5,selfie,False
6,pxg,False
7,pacg,False
8,cct,True
9,escc,True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [15]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [16]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [17]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.1428324

In [18]:
glove_wiki_vectors.similarity("white", "success")

0.351824

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [19]:
# Example 1: Gender bias in occupations
print("Example 1: Gender bias in occupations")
analogy("man", "engineer", "woman")

Example 1: Gender bias in occupations
man : engineer :: woman : ?


,Analogy word,Score
0,technician,0.662020
1,educator,0.611515
2,engineers,0.592235
3,surgeon,0.588020
4,contractor,0.587330
5,engineering,0.568600
6,nurse,0.556342
7,chemist,0.554719
8,pioneer,0.546008
9,physician,0.545960


In [20]:
# Example 2: Cultural/ethnic stereotypes
print("Example 2: Cultural/ethnic biases")
print("Similarity between 'asian' and 'smart':", glove_wiki_vectors.similarity("asian", "smart"))
print("Similarity between 'asian' and 'math':", glove_wiki_vectors.similarity("asian", "math"))
print("Similarity between 'american' and 'smart':", glove_wiki_vectors.similarity("american", "smart"))
print("Similarity between 'american' and 'math':", glove_wiki_vectors.similarity("american", "math"))

Example 2: Cultural/ethnic biases
Similarity between 'asian' and 'smart': 0.19219656
Similarity between 'asian' and 'math': 0.16754796
Similarity between 'american' and 'smart': 0.31898937
Similarity between 'american' and 'math': 0.21283951


In [21]:
# Example 3: Gender roles and personality traits
print("Example 3: Gender roles and personality traits")
print("Similarity between 'woman' and 'emotional':", glove_wiki_vectors.similarity("woman", "emotional"))
print("Similarity between 'man' and 'emotional':", glove_wiki_vectors.similarity("man", "emotional"))
print("Similarity between 'woman' and 'logical':", glove_wiki_vectors.similarity("woman", "logical"))
print("Similarity between 'man' and 'logical':", glove_wiki_vectors.similarity("man", "logical"))

Example 3: Gender roles and personality traits
Similarity between 'woman' and 'emotional': 0.37427944
Similarity between 'man' and 'emotional': 0.44042513
Similarity between 'woman' and 'logical': 0.08263786
Similarity between 'man' and 'logical': 0.18526423


In [22]:
# Example 4: Religious biases
print("Example 4: Religious biases")
print("Similarity between 'muslim' and 'terrorist':", glove_wiki_vectors.similarity("muslim", "terrorist"))
print("Similarity between 'christian' and 'terrorist':", glove_wiki_vectors.similarity("christian", "terrorist"))
print("Similarity between 'muslim' and 'peaceful':", glove_wiki_vectors.similarity("muslim", "peaceful"))
print("Similarity between 'christian' and 'peaceful':", glove_wiki_vectors.similarity("christian", "peaceful"))

Example 4: Religious biases
Similarity between 'muslim' and 'terrorist': 0.51509273
Similarity between 'christian' and 'terrorist': 0.29155728
Similarity between 'muslim' and 'peaceful': 0.43527195
Similarity between 'christian' and 'peaceful': 0.2872671


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

Based on the results from our exploration, several concerning biases exist in the GloVe Wikipedia embeddings:

1. **Gender bias in occupations:**
   The analogy "man is to engineer as woman is to what?" yields "technician" and "educator" at the top, with "nurse" at rank 6. This reveals stereotypical gender-profession associations despite some progress (with "engineers" appearing at rank 3).

2. **Cultural and ethnic stereotypes:**
   Interestingly, "american" shows higher similarity with "smart" (0.319) than "asian" does (0.192), possibly indicating some de-biasing efforts in these embeddings.

3. **Gender and personality traits:**
   Clear bias exists where "man" has higher similarity with "logical" (0.185) than "woman" does (0.083), reinforcing harmful stereotypes. Surprisingly, "man" also has a higher similarity with "emotional" (0.440) than "woman" (0.374).

4. **Religious biases:**
   Most concerning is the significantly higher similarity between "muslim" and "terrorist" (0.515) compared to "christian" and "terrorist" (0.292). Even though "muslim" also has higher association with "peaceful" (0.435), the strong negative association is deeply problematic.

5. **Racial biases:**
   "White" shows much higher similarity with "success" (0.352) than "aboriginal" (0.143), indicating racial bias in attribute associations.

## Real-world harm example:

These biases can cause real harm in applications like:

1. **Content moderation systems:** The high association between "muslim" and "terrorist" could lead to disproportionate flagging of innocent content from Muslim users, effectively silencing minority voices.

2. **Automated recruitment tools:** Gender biases in occupation terms could systematically disadvantage women applying for technical roles. For instance, a resume screening system might rank female candidates lower for engineering positions even when equally qualified, perpetuating gender imbalances in STEM fields.

What makes these biases particularly dangerous is their invisibility within AI systems - they operate behind the scenes while creating the illusion of objective, data-driven decisions. This emphasizes the critical need to audit word embeddings for harmful biases before deployment in real-world applications.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [23]:
from sklearn.datasets import fetch_20newsgroups

In [24]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [25]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [ ]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [30]:
def preprocess_text(text):
    doc = nlp(text.lower())
    
    # Filter tokens: keep only content words, no stopwords, length >= 3
    processed_tokens = []
    for token in doc:
        # Keep only nouns, adjectives, verbs, and adverbs
        if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV']:
            # Filter out stopwords and short tokens
            if not token.is_stop and len(token.text) >= 3:
                # Use lemmatized form
                processed_tokens.append(token.lemma_)
    
    # Join tokens back into a string
    return ' '.join(processed_tokens)

In [ ]:
# Process a small subset first to test
print("Processing a sample of documents...")
df_sample = df.iloc[:5].copy()
df_sample['text_pp'] = [preprocess_text(text) for text in df_sample['text']]
print("Sample preprocessing complete. Example:")
print(df_sample['text_pp'].iloc[0][:100])

Processing a sample of documents...
Sample preprocessing complete. Example:
know read u.s.c sence wonder help u.s.c provide paragraph unlawful person transfer possess machinegu


In [33]:
# Process all documents
print("Processing all documents...")
df['text_pp'] = [preprocess_text(text) for text in df['text']]

# Display a few examples
print("\nExamples of preprocessed text:")
print(df[['text_pp', 'target_name']].iloc[2:6])

Processing all documents...

Examples of preprocessed text:
                                             text_pp             target_name
2  actuallay hand support idea have newsgroup asp...           comp.graphics
3  follow problem bug appreciate help create wind...          comp.windows.x
4  late spokesman tell journalist close air space...   talk.politics.mideast
5  like subscribe leadership magazine wonder disk...  soc.religion.christian


In [34]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late spokesman tell journalist close air space...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,like subscribe leadership magazine wonder disk...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

My preprocessing pipeline uses spaCy to prepare text for topic modeling:

1. **Text normalization:**
   - Converted all text to lowercase for consistency
   - Used spaCy's tokenization to break text into meaningful units

2. **Content filtering with spaCy:**
   - Kept only nouns, adjectives, verbs, and adverbs, which carry the most topical meaning
   - Removed stopwords (common words like "the", "is") that appear frequently but add little topical value
   - Filtered out short tokens (<3 characters) that typically aren't meaningful for topics

3. **Lemmatization:**
   - Applied spaCy's lemmatization to reduce words to their base forms (e.g., "running" → "run")
   - This helps consolidate different forms of the same word, improving topic coherence

This approach focuses on maintaining meaningful terms while removing noise, providing cleaner input for the topic model to identify coherent themes across documents. While it doesn't specifically handle URLs or emails, the stopword and part-of-speech filtering naturally removes many of these elements.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [35]:
from sklearn.decomposition import LatentDirichletAllocation

# Create document-term matrix
vectorizer = CountVectorizer(max_df=0.9, min_df=3, max_features=1000)
X = vectorizer.fit_transform(df['text_pp'])
feature_names = vectorizer.get_feature_names_out()
print(f"Vocabulary size: {len(feature_names)}")

Vocabulary size: 1000


In [36]:
# Helper function to print top words per topic
def print_top_words(model, feature_names, n_top_words=10):
    for topic_idx, topic in enumerate(model.components_):
        top_word_indices = topic.argsort()[:-n_top_words - 1:-1]
        top_words = [feature_names[i] for i in top_word_indices]
        print(f"Topic #{topic_idx}: {' '.join(top_words)}")
    print()

In [37]:
# Try different numbers of topics
for n_topics in [4, 8, 12]:
    print(f"\nFitting LDA model with {n_topics} topics...")
    lda = LatentDirichletAllocation(
        n_components=n_topics,
        max_iter=10,
        learning_method='online',
        random_state=42
    )
    lda.fit(X)
    print(f"Top words for {n_topics} topics:")
    print_top_words(lda, feature_names)


Fitting LDA model with 4 topics...
Top words for 4 topics:
Topic #0: game team year play good win player season turkish get
Topic #1: people say gun know go kill right come state time
Topic #2: file program use window image available include server run system
Topic #3: people think know believe question thing say good time way


Fitting LDA model with 8 topics...
Top words for 8 topics:
Topic #0: say go come people know tell think time want kill
Topic #1: gun law weapon right firearm state people control fire crime
Topic #2: window image available file use server program run display version
Topic #3: believe think people know christian thing say true question mean
Topic #4: know post good time problem book find point read look
Topic #5: armenian turkish people jewish government war state muslim turkey greek
Topic #6: file entry program section output build line include information use
Topic #7: game team play year win player good season get think


Fitting LDA model with 12 topics...


In [38]:
# Based on results, choose 8 topics
n_topics = 8  # This matches our 8 categories
print(f"\nTraining final LDA model with {n_topics} topics...")
final_lda = LatentDirichletAllocation(
    n_components=n_topics,
    max_iter=15,
    learning_method='online',
    random_state=42
)
final_lda.fit(X)


Training final LDA model with 8 topics...


LatentDirichletAllocation(learning_method='online', max_iter=15, n_components=8,
                          random_state=42)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [43]:
print("Topics and their top 10 words:")
print_top_words(final_lda, feature_names, 10)

topic_labels = {}

Topics and their top 10 words:
Topic #0: say go come know people tell think time want happen
Topic #1: gun law weapon right firearm state control fire people crime
Topic #2: window image file use available server program run display application
Topic #3: believe people think know christian thing true say question mean
Topic #4: know post time good problem book find point read look
Topic #5: armenian turkish people jewish government israeli war state attack muslim
Topic #6: file entry program section output build line rule use title
Topic #7: game team play year win player good season get think



In [44]:
# Based on the top words from each topic
topic_labels = {
    0: "General Discussion",
    1: "Gun Control/Rights",
    2: "Computing/Graphics",
    3: "Christianity/Religion",
    4: "Information Sharing",
    5: "Middle East Politics",
    6: "Programming/Documentation",
    7: "Sports",
}

# Print topics with their labels and top words
print("\nLabeled topics with top 10 words:")
for topic_idx, label in topic_labels.items():
    top_words = [feature_names[i] for i in final_lda.components_[topic_idx].argsort()[:-11:-1]]
    print(f"{label}: {', '.join(top_words)}")


Labeled topics with top 10 words:
General Discussion: say, go, come, know, people, tell, think, time, want, happen
Gun Control/Rights: gun, law, weapon, right, firearm, state, control, fire, people, crime
Computing/Graphics: window, image, file, use, available, server, program, run, display, application
Christianity/Religion: believe, people, think, know, christian, thing, true, say, question, mean
Information Sharing: know, post, time, good, problem, book, find, point, read, look
Middle East Politics: armenian, turkish, people, jewish, government, israeli, war, state, attack, muslim
Programming/Documentation: file, entry, program, section, output, build, line, rule, use, title
Sports: game, team, play, year, win, player, good, season, get, think


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [45]:
doc_topic_dists = final_lda.transform(X[:5])

doc_topics_df = pd.DataFrame(doc_topic_dists, 
                           columns=[f"Topic {i}" for i in range(8)])

doc_topics_df['Original Category'] = df['target_name'].iloc[:5].values
doc_topics_df['Dominant Topic'] = [topic_labels[i] for i in doc_topic_dists.argmax(axis=1)]

print("Document-Topic Distribution for the First 5 Documents:")
print(doc_topics_df)

for i in range(5):
    print(f"\nDocument {i}:")
    print(f"Category: {df['target_name'].iloc[i]}")
    print(f"Dominant topic: {topic_labels[doc_topic_dists[i].argmax()]}")
    print(f"Topic distribution: {dict(zip([topic_labels[j] for j in range(8)], doc_topic_dists[i].round(3)))}")

Document-Topic Distribution for the First 5 Documents:
    Topic 0   Topic 1   Topic 2   Topic 3   Topic 4   Topic 5   Topic 6  \
0  0.247838  0.583903  0.003478  0.003478  0.150877  0.003475  0.003475   
1  0.002502  0.002502  0.002502  0.002502  0.982484  0.002502  0.002504   
2  0.003053  0.003051  0.056749  0.147170  0.780825  0.003050  0.003051   
3  0.201713  0.008339  0.298979  0.008346  0.457601  0.008343  0.008342   
4  0.326120  0.005958  0.005970  0.005962  0.005960  0.638108  0.005963   

    Topic 7      Original Category        Dominant Topic  
0  0.003476     talk.politics.guns    Gun Control/Rights  
1  0.002503          comp.graphics   Information Sharing  
2  0.003051          comp.graphics   Information Sharing  
3  0.008337         comp.windows.x   Information Sharing  
4  0.005959  talk.politics.mideast  Middle East Politics  

Document 0:
Category: talk.politics.guns
Dominant topic: Gun Control/Rights
Topic distribution: {'General Discussion': 0.248, 'Gun Control/

## Analysis of Document-Topic Assignments

The LDA model's document-topic assignments demonstrate several key patterns:

1. **Strong category alignment:** Documents generally show highest probability for topics matching their original newsgroup categories, validating the model's effectiveness.

2. **Natural topic mixing:** Most documents exhibit probability across multiple topics, reflecting the reality that authentic texts rarely contain only one theme.

3. **Contextual relationships:** Secondary topic probabilities reveal meaningful connections between different subject areas (e.g., religious discussions with some political aspects).

4. **Concentration variance:** Some documents show strongly concentrated probability in a single topic (focused content), while others distribute more evenly (broader discussions).

5. **Nuanced content representation:** The probabilistic distribution provides a more detailed understanding of document content than simple categorization, capturing thematic subtleties and semantic relationships.

This topic modeling approach successfully reveals both the primary themes and the more subtle thematic connections within the corpus, providing deeper insight than traditional categorization methods.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

### 1. Content-based filtering in recommender systems

Content-based filtering recommends items by matching item features with user preferences. The system:
- Creates profiles of items based on their attributes (genres, keywords, etc.)
- Builds user preference profiles from previously liked items
- Recommends new items with similar features to those the user has liked

Unlike collaborative filtering, it doesn't need data from other users, making it effective for new items and unique preferences.

### 2. Negative consequences of recommender systems

Two key negative consequences of recommender systems:

1. **Filter bubbles/echo chambers:** Systems recommend similar content to what users already consume, limiting exposure to diverse perspectives and potentially increasing polarization.

2. **Algorithmic bias amplification:** Biases in training data become embedded in recommendations, potentially reinforcing harmful stereotypes and discriminatory patterns in content exposure.

### 3. Transfer learning in natural language processing

Transfer learning in NLP involves adapting models pre-trained on large datasets to specific tasks:

1. A model first learns general language patterns from massive text corpora (pre-training)
2. This pre-trained model is then fine-tuned on smaller, task-specific datasets

This approach leverages knowledge gained from large datasets to achieve strong performance on specialized tasks with limited labeled data, dramatically improving efficiency and effectiveness in NLP applications.

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)